## Load in Packages

In [45]:
import pandas as pd
import pymc as pm
import matplotlib.pyplot as plt
import arviz as az
from scipy import special, stats
import numpy as np


## Data Cleaning

In [46]:
# Load in data
state_data = pd.read_csv('https://raw.githubusercontent.com/Mkbonner/BayesianModeling/main/cleaning/State_time_series.csv')
data = pd.read_csv('https://raw.githubusercontent.com/Mkbonner/BayesianModeling/main/cleaning/SAGDP2N__ALL_AREAS_1997_2020.csv')

In [47]:
data.head()

,GeoFIPS,GeoName,Region,TableName,LineCode,IndustryClassification,Description,Unit,1997,1998,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,"""00000""",United States *,,SAGDP2N,1.0,...,All industry total,Millions of current dollars,8577552.0,9062817.0,...,15542582.0,16197007.0,16784851.0,17527258.0,18238301.0,18745075.0,19542980.0,20611861.0,21433226.0,20936558.0
1,"""00000""",United States *,,SAGDP2N,2.0,...,Private industries,Millions of current dollars,7431992.0,7871500.0,...,13405520.0,14037519.0,14572341.0,15255889.0,15898859.0,16360179.0,17094245.0,18062184.0,18793750.0,18290860.0
2,"""00000""",United States *,,SAGDP2N,3.0,11,"Agriculture, forestry, fishing and hunting",Millions of current dollars,108637.0,99756.0,...,180945.0,179573.0,215601.0,201003.0,182283.0,166571.0,176625.0,178569.0,175373.0,175802.0
3,"""00000""",United States *,,SAGDP2N,4.0,111-112,Farms,Millions of current dollars,88136.0,79030.0,...,152249.0,148939.0,184621.0,168147.0,147384.0,130639.0,140053.0,140271.0,136080.0,(NA)
4,"""00000""",United States *,,SAGDP2N,5.0,113-115,"Forestry, fishing, and related activities",Millions of current dollars,20501.0,20726.0,...,28696.0,30634.0,30980.0,32856.0,34899.0,35932.0,36571.0,38298.0,39293.0,(NA)


In [50]:
data.Description.value_counts()

Description
All industry total                                            60
   Professional, scientific, and technical services           60
   Educational services                                       60
  Educational services, health care, and social assistance    60
    Waste management and remediation services                 60
                                                              ..
    Textile mills and textile product mills                   60
All industry total, overseas activity                          1
Government and government enterprises, overseas activity       1
Federal civilian, overseas activity                            1
Military, overseas activity                                    1
Name: count, Length: 96, dtype: int64

In [34]:
# Only want Do not want regions so lets remove them
values_to_exclude = ['Rocky Mountain', 'Southwest', 'Southeast', 'Plains', 'Great Lakes', 'Mideast', 'United States *', 'Far West', 'New England']
mask = ~data['GeoName'].isin(values_to_exclude)
df_filtered = data[mask]

In [35]:
# Drop last 4 row since they are meta data, drop unnecessary columns
dropped = df_filtered.iloc[:-4]
dropped= dropped.drop(columns=['2020', "GeoFIPS", "Unit", 'LineCode','TableName','Region', 'IndustryClassification'])

In [36]:
# Melt the DataFrame so years is a singular column

id_vars = ['GeoName',  'Description']

df_melted = pd.melt(dropped, id_vars=id_vars, var_name='Year', value_name='Value')

In [37]:
# Make sure Value is numeric
df_melted['Value'] = pd.to_numeric(df_melted['Value'], errors='coerce')

In [38]:
# Now we have too many groups, lets pick out the necessary ones

value_counts_result = df_melted['Description'].value_counts()

# Convert the value counts to a list
values_list = value_counts_result.index.tolist()

# Display the list of value
values_list

['All industry total',
 '  Professional and business services',
 '  Educational services, health care, and social assistance',
 '    Waste management and remediation services',
 '    Administrative and support services',
 '   Administrative and support and waste management and remediation services',
 '   Management of companies and enterprises',
 '    Miscellaneous professional, scientific, and technical services',
 '    Computer systems design and related services',
 '    Legal services',
 '   Professional, scientific, and technical services',
 '    Rental and leasing services and lessors of nonfinancial intangible assets',
 ' Private industries',
 '    Real estate',
 '   Real estate and rental and leasing',
 '    Funds, trusts, and other financial vehicles',
 '    Insurance carriers and related activities',
 '    Securities, commodity contracts, and other financial investments and related activities',
 '    Monetary Authorities- central bank, credit intermediation, and related servic

In [39]:
# Only keep rows relatings to these main descriptive features
keep = ['All industry total', '  Agriculture, forestry, fishing and hunting','  Mining, quarrying, and oil and gas extraction', '  Manufacturing',
 '  Construction',
 '  Utilities',  '  Arts, entertainment, recreation, accommodation, and food services', '  Other services (except government and government enterprises)',
  ' Government and government enterprises', '  Finance, insurance, real estate, rental, and leasing', '  Professional and business services',
 '  Educational services, health care, and social assistance',  '  Information','  Transportation and warehousing',
 '  Retail trade',
 '  Wholesale trade'
 ]

filtered_df = df_melted[df_melted['Description'].isin(keep)]


In [40]:
# Now we want these descriptions to be our features so let's make them seperate columns
unmelted_df = filtered_df.pivot(index= ['GeoName', 'Year'], columns='Description', values='Value')

unmelted_df.reset_index(inplace=True)

In [41]:
data = unmelted_df.copy()

In [42]:
# still have too many features, let's condense them even further

data['materials'] = data['  Agriculture, forestry, fishing and hunting'] + data['  Mining, quarrying, and oil and gas extraction']
data['trade']= data['  Retail trade'] + data['  Transportation and warehousing'] + data['  Wholesale trade']
data['public'] = data['  Educational services, health care, and social assistance'] + data[' Government and government enterprises'] + data['  Utilities']
data['service'] = data['  Arts, entertainment, recreation, accommodation, and food services']
data['information'] = data['  Finance, insurance, real estate, rental, and leasing'] + data['  Information'] + data['  Professional and business services']
data['industry'] = data['  Construction'] + data['  Manufacturing']
data['other']= data['  Other services (except government and government enterprises)']

columns_to_keep = ['GeoName', 'Year', 'materials', 'trade', 'public','service','information', 'industry','other']  # Replace with the actual column names you want to keep
df_reindexed = data[columns_to_keep]

#remove any spaces in GeoName column so we can merge with keys of other dataframe
df_reindexed['GeoName'] = df_reindexed['GeoName'].str.replace(' ', '')

/var/folders/x_/qkfj00656_qg6nsj5fclf5sc0000gn/T/ipykernel_67837/1662974652.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reindexed['GeoName'] = df_reindexed['GeoName'].str.replace(' ', '')


In [43]:
df_reindexed

Description,GeoName,Year,materials,trade,public,service,information,industry,other
0,Alabama,1997,3055.9,18202.8,27356.2,2520.7,26501.4,24216.1,2863.7
1,Alabama,1998,3165.1,19484.1,28240.1,2749.1,27461.2,25852.5,3229.2
2,Alabama,1999,3086.4,19902.3,29605.3,3004.1,29781.1,27130.0,3324.0
3,Alabama,2000,2996.5,20595.9,30718.5,3142.9,31443.4,27741.0,3494.7
4,Alabama,2001,3368.6,19899.4,31809.9,3272.0,33962.6,27391.9,3330.9
...,...,...,...,...,...,...,...,...,...
1168,Wyoming,2015,7457.0,7058.7,8895.5,1600.4,7668.9,4565.3,615.1
1169,Wyoming,2016,6566.9,6771.1,8956.7,1580.3,7519.7,3729.3,579.9
1170,Wyoming,2017,7849.8,6751.8,9094.5,1641.5,7381.9,3965.1,586.6
1171,Wyoming,2018,8222.5,7471.5,9296.3,1644.7,8089.6,4349.2,629.4


In [13]:
df = df_reindexed.copy()


# Convert 'year' to integer
df['Year'] = df['Year'].astype(int)

# Set GeoName and year as the index
df.set_index(['GeoName', 'Year'], inplace=True)

# Reindex with a MultiIndex including GeoName and a range of months between the years
idx = pd.MultiIndex.from_product([df.index.levels[0], range(df.index.levels[1].min(), df.index.levels[1].max() + 1), range(1, 13)], names=['GeoName', 'year', 'month'])
df_reindexed = df.reindex(idx)

# Reset index
df_reindexed.reset_index(inplace=True)

columns_to_update = ['materials', 'trade', 'public', 'service','information','industry','other'] 
df_reindexed.loc[df_reindexed['month'] != 1, columns_to_update] = np.nan

# Interpolate using linear method based on 'year' and 'month'
df_interpolated = df_reindexed.groupby('GeoName').apply(lambda group: group.interpolate(method='linear'))

# Now df_interpolated contains interpolated monthly data for each GeoName

subset_df = df_interpolated[df_interpolated['year'] < 2018]

In [14]:
state_economic_data = subset_df.reset_index(drop=True)
state_economic_data

Description,GeoName,year,month,materials,trade,public,service,information,industry,other
0,Alabama,1997,1,3055.900000,18202.800,27356.200000,2520.700000,26501.400000,24216.100000,2863.700000
1,Alabama,1997,2,3065.000000,18309.575,27429.858333,2539.733333,26581.383333,24352.466667,2894.158333
2,Alabama,1997,3,3074.100000,18416.350,27503.516667,2558.766667,26661.366667,24488.833333,2924.616667
3,Alabama,1997,4,3083.200000,18523.125,27577.175000,2577.800000,26741.350000,24625.200000,2955.075000
4,Alabama,1997,5,3092.300000,18629.900,27650.833333,2596.833333,26821.333333,24761.566667,2985.533333
...,...,...,...,...,...,...,...,...,...,...
12847,Wyoming,2017,8,8067.208333,7171.625,9212.216667,1643.366667,7794.725000,4189.158333,611.566667
12848,Wyoming,2017,9,8098.266667,7231.600,9229.033333,1643.633333,7853.700000,4221.166667,615.133333
12849,Wyoming,2017,10,8129.325000,7291.575,9245.850000,1643.900000,7912.675000,4253.175000,618.700000
12850,Wyoming,2017,11,8160.383333,7351.550,9262.666667,1644.166667,7971.650000,4285.183333,622.266667


### This data is clean, now let's clean the other data set

In [15]:
state_data.head()

,Date,RegionName,DaysOnZillow_AllHomes,InventorySeasonallyAdjusted_AllHomes,InventoryRaw_AllHomes,MedianListingPricePerSqft_1Bedroom,MedianListingPricePerSqft_2Bedroom,MedianListingPricePerSqft_3Bedroom,MedianListingPricePerSqft_4Bedroom,MedianListingPricePerSqft_5BedroomOrMore,...,ZHVI_BottomTier,ZHVI_CondoCoop,ZHVI_MiddleTier,ZHVI_SingleFamilyResidence,ZHVI_TopTier,ZRI_AllHomes,ZRI_AllHomesPlusMultifamily,ZriPerSqft_AllHomes,Zri_MultiFamilyResidenceRental,Zri_SingleFamilyResidenceRental
0,1996-04-30,Alabama,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,45600.0,99500.0,79500.0,79000.0,140200.0,NaN,NaN,NaN,NaN,NaN
1,1996-04-30,Arizona,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,67100.0,78900.0,103600.0,107500.0,168700.0,NaN,NaN,NaN,NaN,NaN
2,1996-04-30,Arkansas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,38400.0,70300.0,64400.0,64500.0,115200.0,NaN,NaN,NaN,NaN,NaN
3,1996-04-30,California,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,95100.0,136100.0,157900.0,162000.0,270600.0,NaN,NaN,NaN,NaN,NaN
4,1996-04-30,Colorado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,82700.0,99400.0,128100.0,133600.0,209300.0,NaN,NaN,NaN,NaN,NaN


In [16]:
# Filter the data so the dates match with the previous data set
state_data['Date'] = pd.to_datetime(state_data['Date'])
filtered_df = state_data[(state_data['Date'] > '1996-12-31') & (state_data['Date'] < '2018-01-01')]

In [17]:
# remove anything other than a state
df = filtered_df[filtered_df['RegionName'] != 'UnitedStates']

house_price = df.copy()

In [18]:
house_price['Date'] = pd.to_datetime(house_price['Date'])
columns_to_keep = ['Date', 'RegionName', 'ZHVIPerSqft_AllHomes']

# Drop all columns except the specified ones
house_price= house_price[columns_to_keep]


In [19]:
# we only want year as well so let's drop everything else
house_price['year'] = house_price['Date'].dt.year
house_price['month'] = house_price['Date'].dt.month
house_price = house_price.drop('Date', axis=1)

# Now we are done with data cleaning, let's merge the datasets!

In [23]:
merged_df = pd.merge(house_price, state_economic_data, left_on=['RegionName','year','month'], right_on= ['GeoName','year','month'], how='inner')

#drop extra column
merged_df = merged_df.drop(['RegionName'], axis=1)


In [28]:
# implement a time column based on unique year/time combos
merged_df['instance'] = (12*((merged_df['year'].astype(int))-1997)) + ((merged_df['month'].astype(int)))
merged_df.dropna(inplace=True)

In [29]:
# save merged data
#merged_df.to_csv('/Users/bpugs/Desktop/Bayes/final_project_data_new.csv')